In [1]:
import keras
from keras.preprocessing import image
from keras.engine import Layer
from keras.layers import Conv2D, Conv3D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential, Model, load_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
from time import time
import numpy as np
import os
import random
import tensorflow as tf
from PIL import Image, ImageFile

In [2]:
path = "Downloads/DC3 Dataset/Dataset"
image_name_list = os.listdir(path)[0:10000]
print(len(image_name_list))

10000


In [3]:
from numpy import asarray
X = []
Y = []
for image_name in image_name_list:
    
    img = Image.open(path+'/'+image_name)
    img = img.resize((224, 224))
    img = asarray(img)
    if img.shape == (224, 224):
        print(image_name)
        continue;
    lab = rgb2lab(img)
    lab = lab.astype('float32')
    lab[:,:,0] /= 100.0
    X.append(lab[:,:,0])
    lab[:,:,1:] /= 127.0
    Y.append(lab[:,:,1:])

000000011261.jpg
000000020286.jpg
000000021603.jpg
000000021661.jpg
000000022172.jpg
000000022702.jpg
000000029659.jpg
000000029992.jpg
000000031068.jpg
000000034215.jpg
000000037296.jpg
000000039085.jpg
000000039233.jpg
000000039418.jpg
000000040523.jpg
000000047211.jpg
000000048373.jpg
000000049113.jpg
000000049778.jpg
000000050318.jpg
000000053650.jpg
000000055501.jpg
000000055943.jpg
000000058277.jpg
000000059057.jpg
000000059302.jpg
000000060861.jpg
000000062372.jpg
000000063697.jpg
000000069260.jpg
000000070456.jpg
000000071130.jpg
000000071903.jpg
000000073214.jpg
000000073867.jpg
000000075440.jpg
000000079553.jpg
000000083478.jpg
000000086661.jpg
000000086891.jpg
000000087984.jpg
000000089235.jpg
000000092762.jpg
000000093694.jpg
000000096269.jpg
000000096465.jpg
000000097308.jpg
000000098398.jpg
000000100480.jpg
000000102135.jpg
000000104849.jpg
000000107971.jpg
000000110773.jpg
000000114526.jpg
000000116680.jpg
000000123452.jpg
000000123802.jpg
000000126285.jpg
000000127014.j

In [4]:
X = np.array(X)
Y = np.array(Y)
X = X.reshape(X.shape+(1,))
print(X.shape, X.max(), X.min())
print(Y.shape, Y.max(), Y.min())

(9729, 224, 224, 1) 1.0 0.0
(9729, 224, 224, 2) 0.76082957 -0.8492701


In [5]:
vggmodel = keras.applications.vgg16.VGG16()
newmodel = Sequential() 
num = 0
for i, layer in enumerate(vggmodel.layers):
    if i<19:
      newmodel.add(layer)
newmodel.summary()
for layer in newmodel.layers:
  layer.trainable=False

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [6]:
vggfeatures = []
for i, sample in enumerate(X):
    sample = gray2rgb(sample)
    sample = sample.reshape((1,224,224,3))
    prediction = newmodel.predict(sample)
    prediction = prediction.reshape((7,7,512))
    vggfeatures.append(prediction)
vggfeatures = np.array(vggfeatures)
print(vggfeatures.shape)

(9729, 7, 7, 512)


In [7]:
#Encoder
encoder_input = Input(shape=(7, 7, 512,))
#Decoder
decoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_input)
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
model = Model(inputs=encoder_input, outputs=decoder_output)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 7, 7, 512)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 7, 7, 256)         1179904   
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 128)         295040    
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        73792     
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 32)       

In [40]:
model = load_model('Downloads/colorizationModelVGG.hdf5')

In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint
modelpath = 'Downloads/colorizationModelVGG.hdf5'
checkpoint = ModelCheckpoint(modelpath, monitor='accuracy', verbose=1, save_best_only=True, mode='max')
model.compile(optimizer='Adam', loss='mse' , metrics=['accuracy'])
model.fit(vggfeatures, Y, epochs=100, callbacks = [checkpoint])

Epoch 1/100
304/305 [============================>.] - ETA: 0s - loss: 0.0152 - accuracy: 0.6350
Epoch 00001: accuracy improved from -inf to 0.63506, saving model to Downloads\colorizationModelVGG.hdf5
305/305 [==============================] - 152s 498ms/step - loss: 0.0152 - accuracy: 0.6351
Epoch 2/100
304/305 [============================>.] - ETA: 0s - loss: 0.0128 - accuracy: 0.6543
Epoch 00002: accuracy improved from 0.63506 to 0.65429, saving model to Downloads\colorizationModelVGG.hdf5
305/305 [==============================] - 168s 551ms/step - loss: 0.0128 - accuracy: 0.6543
Epoch 3/100
304/305 [============================>.] - ETA: 0s - loss: 0.0126 - accuracy: 0.6579
Epoch 00003: accuracy improved from 0.65429 to 0.65787, saving model to Downloads\colorizationModelVGG.hdf5
305/305 [==============================] - 174s 571ms/step - loss: 0.0126 - accuracy: 0.6579
Epoch 4/100
304/305 [============================>.] - ETA: 0s - loss: 0.0124 - accuracy: 0.6611
Epoch 00004:

In [9]:
L_part = []
AB = []
for image_name in os.listdir("Downloads/test/"):
    test_image = Image.open("Downloads/test/"+image_name)
    test_image = test_image.resize((224, 224))
    test_image = asarray(test_image)
    print(image_name)
    try:
        lab_test = rgb2lab(test_image)
    except:
        test_gray = gray2rgb(test_image)
        lab_test = rgb2lab(test_gray)
    lab_test = lab_test.astype('float32')
    lab_test[:,:,0] /= 100
    L_part.append(lab_test[:,:,0])
    L = gray2rgb(lab_test[:,:,0])
    L = gray2rgb(lab_test[:,:,0])
    L = L.reshape((1,224,224,3))
    vggpred = newmodel.predict(L)
    ab = model.predict(vggpred)
    ab *= 127
    AB.append(ab)
print(len(AB))

2016-06-27-black-white-photo-06.jpg
22art2-1.jpg
dia mirza.jpg
elephant.jpg
images (1).jfif
images (2).jfif
images.jfif
lion.jpg
main-qimg-52365feb99da5b9bce6ae216c18e6660.jfif
matt_miller_embed-710x1066.jpg
missing-you-black-white-16655.jpg
p-364-jackanstey-03283-jj.jpg
photo-1501059520966-c59141726d6c.jfif
road.jpg
xovk0byrmkcuvn0vjnyoxg.jpg
15


In [11]:
for i in range(len(AB)):
    cur = np.zeros((224, 224, 3))
    cur[:,:,0] = L_part[i]*100
    cur[:,:,1:] = AB[i]
    rgb = lab2rgb(cur)
    rgb *= 255
    rgb = rgb.astype('uint8')
    imsave("Downloads/results3/result_vgg"+str(i)+".png", rgb)